In [8]:
# Import Packages and setup
from __future__ import print_function, division
import os
from collections import OrderedDict
import logging

import numpy as np

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
# Make the plots look pretty
matplotlib.rc('font',size=13)
matplotlib.rc('legend', numpoints=1)
matplotlib.rc('image', cmap='inferno')
matplotlib.rc('image', interpolation='none')
# Use the same color cylcer as Seaborn
from cycler import cycler
color_cycle = [u'#4c72b0', u'#55a868', u'#c44e52', u'#8172b2', u'#ccb974', u'#64b5cd']
matplotlib.rcParams['axes.prop_cycle'] = cycler("color", color_cycle)

import lsst.afw.table as afwTable
from lsst.utils import getPackageDir
import lsst.log as log
import lsst.meas.deblender
from lsst.meas.deblender import proximal, display, sim, baseline
import lsst.meas.deblender.utils as debUtils
from lsst.afw.detection import Footprint

logger = logging.getLogger("lsst.meas.deblender")
logger.setLevel(logging.INFO)
log.setLevel("", log.INFO)
plogger = logging.getLogger("proxmin")
plogger.setLevel(logging.INFO)

In [9]:
import pickle
expDb = pickle.load(open("/Users/fred/temp/expDb1.p", "rb"))
simCat, simTable = sim.loadSimCatalog(expDb.simCat)
#expDb.simCat = simCat
#expDb.simTable = simTable

In [10]:
parent_indices = list(expDb.deblends['new'].keys())
#sim.compareDeblendToSim(expDb.deblends["new"][parent_indices[0]], expDb.mergedDet[0], simTable)

In [11]:
deblend = expDb.deblends["new"][parent_indices[0]]
avgNoise = sim.getNoise(expDb.calexps)
peakTable = sim.buildFootprintPeakTable(deblend.footprint, deblend.filters, peaks=deblend.peaks)

In [9]:
def matchFootprintToRef(footprint, peakTable, simTable, filters, maxSeparation=3, poolSize=-1, avgNoise=None,
                        display=True, calexp=None, bbox=None, expDb=None):
    """Match a peakTable for a single Footprint to a catalog

    See ``matchToRef`` for a description of parameters, other than ``Footprint``, which is the
    `lsst.afw.detection.Footprint` used to get the bounds for the simTable and return
    sources not contained in the ``peakTable``, and ``expDb``, which is only needed to
    to display the image (when ``display=True``).
    """
    bbox = footprint.getBBox()
    xmin = bbox.getMinX()
    xmax = bbox.getMaxX()
    ymin = bbox.getMinY()
    ymax = bbox.getMaxY()

    return
    simCuts = (simTable["x"]>=xmin)&(simTable["x"]<=xmax) & (simTable["y"]>=ymin) & (simTable["y"]<=ymax)
    logger.info(np.sum(simCuts))

    if avgNoise is None:
        logger.info("calculating avg noise")
        # Estimate the noise in the image
        avgNoise = getNoise(expDb.calexps)
    # Match the simulated data to the detected peaks
    matchTable, idx, unmatchedTable = matchToRef(peakTable, simTable[simCuts], filters, display=False)

    if display:
        x = list(range(len(filters)))
        for pk, src in enumerate(matchTable):
            flux = np.array([np.max(src["intensity_{0}".format(f)]) for f in filters])
            plt.semilogy(x, flux, 'k.-', label="Detected")
        for src in unmatchedTable:
            flux = np.array([np.max(src["intensity_{0}".format(f)]) for f in filters])
            plt.plot(x, flux, '.-', c="#4c72b0", label="Not Detected")
        if avgNoise is not None:
            plt.semilogy(x, avgNoise, '.-', c="#c44e52", label="Background")
            plt.semilogy(x, 2*np.array(avgNoise), '.--', c="#c44e52", label="2$\\times$Background")
        plt.legend(loc='center left', bbox_to_anchor=(1, .5),
                   fancybox=True, shadow=True)
        plt.xticks([-.25]+x+[x[-1]+.25], [""]+[f for f in filters]+[""])
        plt.xlabel("Filter")
        plt.ylabel("Peak Flux")
        plt.show()

        fig = plt.figure(figsize=(8,8))
        ax = fig.add_subplot(1,1,1)
        calexps = [calexp[footprint.getBBox()] for calexp in expDb.calexps]
        ax = debDisplay.plotColorImage(calexps=calexps, ax=ax, show=False)
        px = [src["x"]-xmin for src in unmatchedTable]
        py = [src["y"]-ymin for src in unmatchedTable]
        plt.plot(px, py, "rx", mew=2, label="Not detected peaks")
        px = [src["x"]-xmin for src in matchTable]
        py = [src["y"]-ymin for src in matchTable]
        ax.plot(px, py, "cx", mew=2, label="Detected peaks")
        plt.legend(loc="center left", fancybox=True, shadow=True, ncol=1, bbox_to_anchor=(1, 0.5))
        # Sometimes, if sources are near the edge, the plot extends beyond the image size
        plt.xlim([0,footprint.getBBox().getWidth()-1])
        plt.ylim([footprint.getBBox().getHeight()-1, 0])
        plt.show()

    return matchTable, idx, unmatchedTable

In [ ]:
matchFootprintToRef(deblend.footprint, peakTable, simTable,
                                     deblend.filters, avgNoise=avgNoise,display=False)

In [ ]:
deblend.footprint.getBBox()

In [ ]:
deblend.footprint.getPeaks()

In [7]:
dir(deblend.footprint)

['__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'addPeak',
 'assign',
 'census',
 'clipTo',
 'contains',
 'dilate',
 'erode',
 'getArea',
 'getBBox',
 'getCentroid',
 'getId',
 'getNextMemId',
 'getPeaks',
 'getRegion',
 'getShape',
 'getSpans',
 'isContiguous',
 'isHeavy',
 'isPersistable',
 'markPersistent',
 'peaks',
 'readFits',
 'removeOrphanPeaks',
 'repr',
 'setDeleteCallbackId',
 'setNewCallbackId',
 'setPeakSchema',
 'setRegion',
 'setSpans',
 'shift',
 'sortPeaks',
 'spans',
 'split',
 'transform',
 'writeFits']